# Filtracja sygnału audio
## Przedmiot: Systemy Dedykowane w Układach Programowalnych
## Autorzy: Paweł Olbrych, Michał Zelek

## Filtry


Filtrację dźwięku można zrealizować jako **bank filtrów** — zestaw filtrów połączonych równolegle lub szeregowo i różniących się np. częstotliwością środkową czy typem[^eq-wiki]. System taki pozwala kształtować charakterystykę częstotliwościową sygnału zgodnie z potrzebami.

- **Filtr peak (band-pass/boost-cut)** służy do zmiany poziomu amplitudy w zadanym paśmie.  
- **Filtr notch (band-stop)** eliminuje wybrane składowe.  

<!-- Rysunki peak_filter.png i notch_filter.png -->
<img src="img/peak_filter.png" style="width:40%;">
<!-- ![Charakterystyka filtru peak](img/peak_filter.png)   -->
*Rys. 1. Charakterystyka filtru peak (oprac. własne).*

<img src="img/notch_filter.png" style="width:40%;">
<!-- ![Charakterystyka filtru notch](img/notch_filter.png)   -->
*Rys. 2. Charakterystyka filtru notch (oprac. własne).*

Parametry filtrów **peak** i **notch**:

- $$f\_0$$ – częstotliwość środkowa  
- $$A$$ – wartość wzmocnienia  
- $$BW$$ – pasmo (szerokość mierzoną przy \(A{-}3\) dB; dla notch zawsze \(-3\) dB)

Obydwa są filtrami IIR 2-go rzędu, charakteryzującymi się dużą selektywnością.

---

## Projekt filtru IIR

W procesie projektowania często korzysta się z protokołu analogowego, np. filtru LC z rys. 3:

![Filtr pasmowo-zaporowy LC](img/Band-Reject_Filter.svg)  
<!-- <img src="img/Band-Reject_Filter.svg" style="width:30%;"> -->
*Rys. 3. Analogowy filtr band-stop[^br-svg].*

Transmitancja obwodu:

$$
H(s)=\frac{V_o}{V_i},
\qquad
s=j2\pi f=j\omega.
$$

Impedancje elementów:

$$
X_C=\frac{1}{j\omega C}=\frac{1}{sC},
\qquad
X_L=j\omega L=sL.
$$

Stosując prawo Kirchhoffa otrzymujemy  

$$
\frac{V_i-V_o}{R}=\frac{V_o}{X_L+X_C}.
$$

Po uproszczeniu:

$$
H(s)=\frac{s^{2}+\dfrac{1}{LC}}%
        {s^{2}+s\dfrac{R}{L}+\dfrac{1}{LC}}
     =\frac{s^{2}+\omega_0^{2}}%
           {s^{2}+s\omega_w+\omega_0^{2}},
$$

gdzie \(\omega_0\) – pulsacja środkowa,  
\(\omega_w\) – pulsacja pasma.

### Transformacja biliniowa

$$
s=\frac{2}{T}\,\frac{z-1}{z+1}
$$

pozwala przejść do dziedziny cyfrowej:

$$
H(z)=\frac{b_0+b_1z^{-1}+b_2z^{-2}}%
           {1+a_1z^{-1}+a_2z^{-2}}
     =\frac{Y(z)}{X(z)}.
$$

Równanie różnicowe (DF-I):

$$
y[n]=b_0x[n]+b_1x[n-1]+b_2x[n-2]
      -a_1y[n-1]-a_2y[n-2].
$$

![Schemat blokowy filtru IIR](img/Biquad_filter_DF-I.svg)  
*Rys. 4. Struktura biquada DF-I[^biquad-svg].*

### Współczynniki filtrów peak / notch

<table>
  <thead><tr><th>Współczynnik</th><th>Wzór (blokowy)</th></tr></thead>
  <tbody>
    <tr><td><em>a</em><sub>0</sub></td>
        <td>$$4 + 2\frac{g}{Q}\,\omega_c T + (\omega_c T)^2$$</td></tr>
    <tr><td><em>a</em><sub>1</sub></td>
        <td>$$\displaystyle\frac{2\bigl((\omega_c T)^2 - 8\bigr)}{a_0}$$</td></tr>
    <tr><td><em>a</em><sub>2</sub></td>
        <td>$$\displaystyle\frac{4 - 2\frac{g}{Q}\,\omega_c T + (\omega_c T)^2}{a_0}$$</td></tr>
    <tr><td><em>b</em><sub>0</sub></td>
        <td>$$\displaystyle\frac{4 + 2\frac{1}{Q}\,\omega_c T + (\omega_c T)^2}{a_0}$$</td></tr>
    <tr><td><em>b</em><sub>1</sub></td>
        <td>$$\displaystyle\frac{-2\bigl((\omega_c T)^2 - 8\bigr)}{a_0}$$</td></tr>
    <tr><td><em>b</em><sub>2</sub></td>
        <td>$$-\displaystyle\frac{4 - 2\frac{1}{Q}\,\omega_c T + (\omega_c T)^2}{a_0}$$</td></tr>
  </tbody>
</table>

Parametry:  

* $$g$$ – liniowa wartość wzmocnienia  
* $$Q$$ – dobroć filtru  
* $$T$$ – okres próbkowania  
* $$\omega_c$$ – pulsacja środkowa

---

[^octave-wiki]: <https://pl.wikipedia.org/wiki/Oktawa_(akustyka)>
[^eq-wiki]: <https://pl.wikipedia.org/wiki/Equalization>
[^br-svg]: Źródło grafiki: Wikipedia, _Band-Reject Filter.svg_ (CC-BY-SA).
[^biquad-svg]: Źródło grafiki: Wikipedia, _Digital Biquad Filter DF-I.svg_ (CC-BY-SA).


In [153]:
from pynq import Overlay
# zybo_sdup_ov = Overlay("design_AUDIO_AXI.xsa")
zybo_sdup_ov = Overlay("design_AdjustCoeffsAXI_wrapper.xsa")

In [15]:
zybo_sdup_ov?


In [2]:
from pynq import MMIO

mmio = MMIO(base_addr=0x40000000, length=0x1000)  # 4 kB

In [13]:
mmio.write(0x00, 0b01)
# this stalls the kernel possiblly AXI signals issue

In [14]:
mmio.read(0x00)
# this stalls the kernel possiblly AXI signals issue

1

In [13]:
print("siema tu zybo")

siema tu zybo


In [7]:
base_addr = zybo_sdup_ov.AudioProcessingSyste_0
print(base_addr)

In [9]:
mmio.write(0x00, 0b01)

In [10]:
mmio.write(0x00, 0b00)

In [21]:
import json
from pathlib import Path
from typing import Union, Dict, Any
from pynq import MMIO

def float_to_fixed_point(value: float, decimal_bits: int = 30) -> int:
    scaling_factor = 2 ** decimal_bits
    signed_32 = int(round(value * scaling_factor)) & 0xFFFFFFFF
    return signed_32       

REG_OFFSET = {           # coefficient → indeks 0-4
    "b0": 0,   # reg  0–7
    "b1": 1,   # reg  8–15
    "b2": 2,   # reg 16–23
    "a1": 3,   # reg 24–31
    "a2": 4    # reg 32–39
}
def load_eq_from_json(json_path: str,
                      mmio: MMIO,
                      enable_eq: bool = True,
                      base_addr: int = 0x00) -> None:
    """reads filter_configuration.json → 41 registers AXI (0–40)."""
    filters = json.loads(Path(json_path).read_text())["filters"]
    if len(filters) > 8:
        raise ValueError("There is max 8 filters, found: %d" % len(filters))

    for section, f in enumerate(filters):
        coeffs = {
            "b0": f["coefficients"]["b"]["b0"],
            "b1": f["coefficients"]["b"]["b1"],
            "b2": f["coefficients"]["b"]["b2"],
            "a1": f["coefficients"]["a"]["a1"],
            "a2": f["coefficients"]["a"]["a2"],
        }
        for name, val in coeffs.items():
            reg_idx = section + 8 * REG_OFFSET[name]   # 0-39
            addr    = base_addr + reg_idx * 4
            mmio.write(addr, float_to_fixed_point(val))

    # rejestr 40 – control_signal
    mmio.write(base_addr + 40 * 4, 0x01 if enable_eq else 0x00)


In [18]:
ip_name = "myip_v1_0_S00_AXI_0"
ip_info = zybo_sdup_ov.ip_dict[ip_name]
mmio = MMIO(ip_info["phys_addr"], 0xA4)

In [20]:
print(ip_info["phys_addr"])

1073741824


In [142]:
load_eq_from_json("filters/filter_configuration.json", mmio, enable_eq=True)

Odtwarzanie dzwięku


<audio controls>
  <source src="audio_samples/music_sample-s.mp3" type="audio/mpeg">
    Twoja przeglądarka nie obsługuje odtwarzania MP3.
</audio>

In [159]:
# kaskada filtrów wzmamciających niskie częstotliwości 
load_eq_from_json("filters/bass_enhancer_filters.json", mmio, enable_eq=True)

In [160]:
# kaskada filtrów tłumiących niskie częstotliwości 
load_eq_from_json("filters/bass_attenuate_filters.json", mmio, enable_eq=True)

Wyłączenie filtrów

In [163]:
#BYPASS brak filtracji sygnału
mmio.write(0x00 + 40 * 4, 0x00)

Odtwarzanie dzwięku ze zniekształceniem sinusoidalnym o częstotliwości 2 kHz


<audio controls>
  <source src="audio_samples/music_sample_2k_distortion-s.mp3" type="audio/mpeg">
</audio>

In [162]:
# kaskada filtrów tłumiących częstotliwość 2kHz z Q=1
load_eq_from_json("filters/2k_attenuate_filters.json", mmio, enable_eq=True)

Odtwarzanie dzwięku ze zniekształceniem sinusoidalnym o częstotliwości 5 kHz


<audio controls>
  <source src="audio_samples/music_sample_2k_distortion-s.mp3" type="audio/mpeg">
</audio>

filtracja ze współczynnikiem dobroci Q=1, filtr z pasmem 500Hz

In [155]:
# kaskada filtrów tłumiących częstotliwość 5kHz z Q=1
load_eq_from_json("filters/5k_atteunate_filters.json", mmio, enable_eq=True)

filtracja ze współczynnikiem dobroci Q=5, filtr z pasmem 100Hz

In [154]:
# kaskada filtrów tłumiących częstotliwość 5kHz z większą selektywnością tj. Q=5, => pasmo filtru = f_filtra/Q => Pasmo = 5000/5 = 100Hz
load_eq_from_json("filters/5k_attenuate_filters_narrow.json", mmio, enable_eq=True)

In [151]:
import wave, contextlib
f='audio_samples/music_sample.wav'
with contextlib.closing(wave.open(f)) as w: 
    print(w.getsampwidth()*8,'bit',w.getframerate(),'Hz')

16 bit 44100 Hz
